# Details

**Name:** Richard Hitchens
    
**Email:** richard.hitchens@berkeley.edu

**Class Name:** W261-1

**Week Number:** 1

**Date:** 9:03pm PST 9 May 2017  

# DATASCI W261: Machine Learning at Scale

# This notebook provides a poor man Hadoop through command-line and python. Please insert the python code by yourself.

# Map

In [1]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+") # this does nothing in my code
filename = sys.argv[2]
findword = sys.argv[1].lower() # Using 'lower()' to make case insensitive

# for each 'line' in the chunk of the file sent to the mapper, i.e. 'myfile'
# check if 'findword' is in the 'line' (also converted to 'lower()')
# if so add 1 to the aggregate 'count' and if no 'findword' is found in the
# chunk then zero is returned
with open (filename, "r") as myfile:
    for line in myfile:
        if findword in line.lower():
            count += 1
print count

# Note that this doesn't count multiple occurrences of 'findword' in a 'line'.
# It only counts 1 if there is at least one instance found in a given 'line',
# i.e. a line counter mapper function.



Overwriting mapper.py


In [2]:
!chmod a+x mapper.py

# Reduce

In [3]:
%%writefile reducer.py
#!/usr/bin/python
import sys
sum = 0
# convert the value of each 'line', i.e. the numeric result of the mapper function
# on each chunk, from a string to an integer and sum them to get the aggregate
# number of lines with 'findword' in them  
for line in sys.stdin:
    sum += int(line)
print sum

Overwriting reducer.py


In [4]:
!chmod a+x reducer.py

# Write script to file

In [5]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Overwriting pGrepCount.sh


# Run the file

In [6]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [7]:
!./pGrepCount.sh License.txt COPYRIGHT 4k

found [57] [COPYRIGHT] in the file [License.txt]
